In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from abides_core import abides
from abides_core.utils import parse_logs_df, ns_date, str_to_ns, fmt_ts
from abides_markets.configs import prmsc1, prmsc2

In [4]:
config = prmsc2.build_config()

In [ ]:
config.keys()

In [ ]:
end_state = abides.run( config )

# Metric Slice

In [7]:
time_mesh = np.arange(
    config["start_time"],
    config["stop_time"],
    1e9*60*60  # Show per 1h
)

In [8]:
time_mesh = time_mesh[1:]
max_time = time_mesh[-1]
trading_agents = end_state["agents"][1:]
num_agent = len(trading_agents)

metric_table = np.zeros((num_agent, len(time_mesh), 3))

for i, agent in enumerate(trading_agents):
    j = 0
    last_metric = [0, np.inf, np.inf]
    
    for event in agent.log:
        if event[1] == 'METRIC':
            if event[0] >= max_time:
                continue
            while event[0] > time_mesh[j]:
                metric_table[i, j] = last_metric
                j += 1
            metric_table[i, j] = np.array(event[2])  # Metric of agent i at time j: [Volume, R1, R2]
            last_metric = np.array(event[2])

In [9]:
def metric_slice(table, t, plot=False):
    metric_slice = table[:, t, :]

    metric_long = [metric for metric in metric_slice if metric[0] > 0]
    metric_long = sorted(metric_long, key=lambda x: -x[1])
    
    metric_short = [metric for metric in metric_slice if metric[0] < 0]
    metric_short = sorted(metric_short, key=lambda x: x[1])

    # Long orders
    vol_long = [metric[0] for metric in metric_long]
    RG1_long = [metric[1] for metric in metric_long]
    RG2_long = [metric[2] for metric in metric_long]

    cum_vol_long = np.cumsum(vol_long)
    total_vol_long = cum_vol_long[-1] if len(cum_vol_long) > 0 else 0

    # Short orders
    vol_short = [-metric[0] for metric in metric_short]
    RG1_short = [metric[1] for metric in metric_short]
    RG2_short = [metric[2] for metric in metric_short]

    cum_vol_short = np.cumsum(vol_short)
    total_vol_short = cum_vol_short[-1] if len(cum_vol_short) > 0 else 0

    # print(total_vol_long, total_vol_short)
    if (total_vol_long != total_vol_short):
        print(f"{total_vol_long}, {total_vol_short}", t)

    if plot:
        plt.plot(cum_vol_short, RG2_short, label = "RG2Short")
        plt.plot(cum_vol_short, RG1_short, label = "RG1Short")
        plt.plot(cum_vol_long, RG1_long, label = "RG1Long")
        plt.plot(cum_vol_long, RG2_long, label = "RG2Long")

        plt.xlabel("Notional")
        plt.ylabel("Tick")
        plt.title(f"Global Metric at time {t}")
        
        plt.ylim(0, 3000)
        plt.legend()
        plt.show()

    return {"OI": total_vol_long, 
            "VolL": cum_vol_long, "VolS": cum_vol_short, 
            "RG1L": RG1_long, "RG2L": RG2_long, 
            "RG1S": RG1_short, "RG2S": RG2_short}

In [ ]:
metric_slice(metric_table, 100, plot=True)

# Metric Summary

## Price Impact

In [11]:
def price_impact(book_log):
    bids = book_log["bids"]
    asks = book_log["asks"]
    time = book_log["QuoteTime"]
    bid_price = [bid[0] for bid in bids]
    bid_vol = np.cumsum([bid[0] for bid in bids])

    ask_price = [ask[0] for ask in asks]
    ask_vol = np.cumsum([ask[0] for ask in asks])

    plt.plot(bid_vol, bid_price, label="BID")
    plt.plot(ask_vol, ask_price, label="ASK")
    plt.title(f"Price Impact at time {fmt_ts(time)}")
    plt.show()

In [12]:
order_book = end_state["agents"][0].order_books["PEN"]
twap = pd.DataFrame(order_book.twap_record,columns=["time","price"])

In [ ]:
end_state["agents"][0].order_books["PEN"].book_log2[100000]

In [ ]:
price_impact(end_state["agents"][0].order_books["PEN"].book_log2[100000])

## RGs

In [15]:
max_time = time_mesh[-1]

small_twap = np.zeros(len(time_mesh))

j = 0

for twap_by_time in twap.itertuples():
    if twap_by_time[1] >= max_time:
        break
    while twap_by_time[1] > time_mesh[j]:
        j += 1
    small_twap[j] = twap_by_time[2]
    
small_twap = small_twap[1:] 

In [16]:
def RG_by_vol(S, cum_vol, R_array):
    for i, cum_vol in enumerate(cum_vol):
        if cum_vol > S:
            return R_array[i]

### RG1

In [ ]:
R1_summary = np.zeros((metric_table.shape[1], 4))

for i in range(metric_table.shape[1]):
    slice = metric_slice(metric_table, i)
    R1_summary[i, :] = [RG_by_vol(0.10*slice["OI"], slice["VolL"], slice["RG1L"]), 
                        RG_by_vol(0.20*slice["OI"], slice["VolL"], slice["RG1L"]),
                        RG_by_vol(0.10*slice["OI"], slice["VolL"], slice["RG1S"]),
                        RG_by_vol(0.20*slice["OI"], slice["VolL"], slice["RG1S"])
    ]

plt.ylim(0, 3000)
plt.plot(R1_summary, label={"RG1Long_10",
                            "RG1Long_20",
                            "RG1Short_10",
                            "RG1Short_20"})

plt.plot(small_twap, label="TWAP")

plt.title("R1 summary")
plt.legend()
plt.show()

### RG2

In [ ]:
R2_summary = np.zeros((metric_table.shape[1], 4))

for i in range(metric_table.shape[1]):
    slice = metric_slice(metric_table, i)
    R2_summary[i, :] = [RG_by_vol(0.10*slice["OI"], slice["VolL"], slice["RG2L"]), 
                        RG_by_vol(0.20*slice["OI"], slice["VolL"], slice["RG2L"]),
                        RG_by_vol(0.10*slice["OI"], slice["VolL"], slice["RG2S"]),
                        RG_by_vol(0.20*slice["OI"], slice["VolL"], slice["RG2S"])
    ]

plt.ylim(0, 3000)
plt.plot(R2_summary, label={"RG2Long_10",
                            "RG2Long_20",
                            "RG2Short_10",
                            "RG2Short_20"})
plt.plot(small_twap, label="TWAP")

plt.title("R2 summary")
plt.legend()
plt.show()

# One value agent

In [19]:
agent = end_state["agents"][1041]

In [ ]:
time_index_R1 = []
R1_val = []

for event in agent.log:
    if event[1] == 'R1':
        time_index_R1.append(event[0])
        R1_val.append(event[2])

plt.plot(time_index_R1, R1_val, label="R1")
time_mesh = np.arange(
    config["start_time"],
    config["stop_time"],
    1e9*60*60*24*15  # Show per 5 days
)
_=plt.xticks(time_mesh, [ fmt_ts(time).split(" ")[0] for time in time_mesh], rotation=60 )

In [ ]:
time_index_R2 = []
R2_val = []

for event in agent.log:
    if event[1] == 'R2':
        if event[2][0] > 0 and event[2][0] < 2000:
            time_index_R2.append(event[0])
            R2_val.append(event[2][0])

plt.plot(time_index_R2, R2_val, label="R2")
time_mesh = np.arange(
    config["start_time"],
    config["stop_time"],
    1e9*60*60*24*5  # Show per 5 days
)
_=plt.xticks(time_mesh, [ fmt_ts(time).split(" ")[0] for time in time_mesh], rotation=60 )

# Log observation

In [22]:
order_book = end_state["agents"][0].order_books["PEN"]

In [23]:
logs_df = parse_logs_df( end_state )

In [ ]:
print(logs_df[ (logs_df.agent_type == "ValueAgent")  ].to_string())

In [ ]:
print(logs_df[ (logs_df.agent_type == "LiquidatorAgent")  ].to_string())

# Orderbook

In [26]:
L1 = order_book.get_L1_snapshots()

In [ ]:
best_bids = pd.DataFrame(L1["best_bids"],columns=["time","price","qty"])
best_asks = pd.DataFrame(L1["best_asks"],columns=["time","price","qty"])
twap = pd.DataFrame(order_book.twap_record,columns=["time","price"])

plt.plot(best_bids.time,best_bids.price, label="BID")
plt.plot(best_asks.time,best_asks.price, label="ASK")
plt.plot(twap.time,twap.price, label="TWAP")
plt.legend()

time_mesh = np.arange(
    config["start_time"],
    config["stop_time"],
    1e9*60*60*24*15  # Show per 15 days
)
_=plt.xticks(time_mesh, [ fmt_ts(time).split(" ")[0] for time in time_mesh], rotation=60 )

In [28]:
transaction_time_mesh = np.arange(
    config["start_time"],
    config["stop_time"] + 1,
    1e9*60*60*24  # Show per 1 day
)
transaction_time_mesh = transaction_time_mesh[1:]  # Drop the first timestamp

In [29]:
buys = pd.DataFrame(order_book.buy_transactions,columns=["time","price","qty"])
sells = pd.DataFrame(order_book.sell_transactions,columns=["time","price","qty"])

buys_pday = np.zeros(len(transaction_time_mesh))
counter = 0
for buy in buys.itertuples():
    if buy.time > transaction_time_mesh[counter]:
        counter += 1
    if counter >= len(transaction_time_mesh):
        break
    buys_pday[counter] += buy.qty

sells_pday = np.zeros(len(transaction_time_mesh))
counter = 0
for sell in sells.itertuples():
    if sell.time > transaction_time_mesh[counter]:
        counter += 1
    if counter >= len(transaction_time_mesh):
        break
    sells_pday[counter] += sell.qty

In [ ]:
plt.plot(transaction_time_mesh, buys_pday, label="BUY volume")
plt.plot(transaction_time_mesh, sells_pday, label="SELL volume")
plt.legend()
plt.xticks(time_mesh, [ fmt_ts(time).split(" ")[0] for time in time_mesh], rotation=60 )
plt.show()